#### Quantile Regression

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
from datetime import datetime, timedelta, date, time
import calendar
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
color_pal = sns.color_palette()

#### Set parameters

In [2]:
# define model name for result files
model_name = "Quantile_Regression"  

# define quantiles 
quantiles = [0.025, 0.25, 0.5, 0.75, 0.975]

# set parametrs for model training and evaluation
test_start_date = "2024-02-22"  # test/evaluation data start
test_size = 168                 # prediction intervall for one split (168 hours = 1 week)
n_splits = 52                    # number of splits for TimeSeriesSplit

# Set Quantil-training data size in hours which is used for evaluation and final prediction
train_windows = {
    "q1_train_window": 8760 * 2,  # hours
    "q2_train_window": 8760 * 2,  # hours
    "q3_train_window": 8760 * 2,  # hours
    "q4_train_window": 8760 * 2,  # hours
    "q5_train_window": 8760 * 2,  # hours
}

# Switch features on or off
time_based_features = 1         # 0 for no time based features, 1 for dummy variables, 2 for categorical variables
lag_1week = 1                   # 0 for no lag features, 1 for lag features
lag_2week_mean = 1              # 0 for no lag features, 1 for lag features
lag_4week_mean = 1             # 0 for no lag features, 1 for lag features

#### Load Data

In [3]:
df = pd.read_csv('../data/combined_data/combined_energy_data.csv', parse_dates=["Datetime"], index_col="Datetime")

# define time zone as ezrope/berlin to account for time shifts in original data
df.index = pd.to_datetime(df.index, utc=True).tz_convert('Europe/Berlin')

# convert back to utc to rmeove time shifts
df.index = df.index.tz_convert('UTC')

# remove tz awareness
df.index = df.index.tz_localize(None)

KeyboardInterrupt: 

#### Prepare and Clean Data for Model

In [ ]:
df = df.loc['2016-01-01':]
df.drop_duplicates(inplace=True)
nan_count = df['target'].isna().sum()
nan_indices = df[df['target'].isna()].index
full_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq='H')
missing_hours = full_range.difference(df.index)
duplicates = df[df.duplicated()]

print(f"NaN count in target column: {nan_count}")
print("Index", nan_indices)
print("Missing hours:", missing_hours)
print("Duplicates:", duplicates)

Anzahl der NaN-Werte in der target-Spalte: 0
Indizes der NaN-Werte in Spalte 'A': DatetimeIndex([], dtype='datetime64[ns]', name='Datetime', freq=None)
Fehlende Stunden: DatetimeIndex([], dtype='datetime64[ns]', freq='H')
Duplikate: Empty DataFrame
Columns: [ghi, rain, target, temperature, wind_speed_100m, wind_speed_10m, public_holiday]
Index: []


#### Create Time Based Features and Lag Features

In [ ]:
# function for time based dummy features
if time_based_features == 1:
    def create_time_dummies(df):

        df['hour'] = df.index.hour.astype(int)
        df['dayofweek'] = df.index.dayofweek.astype(int)
        df['month'] = df.index.month.astype(int)
        df['year'] = df.index.year.astype(int)
        
        df_dummies = pd.get_dummies(df, columns=['hour', 'dayofweek', 'month', 'year'], dtype=int, drop_first=True)
        
        return df_dummies

    df = create_time_dummies(df)

# function for categorical time based features
if time_based_features == 2:
    
    def create_time_features(df):
        df = df.copy()
        df['hour'] = df.index.hour
        df['dayofweek'] = df.index.dayofweek
        df['month'] = df.index.month
        df['year'] = df.index.year
        df['weekofyear'] = df.index.isocalendar().week.astype(int)
    
        return df

    df = create_time_features(df)

In [ ]:
# Lag 1 week, mean 2 week, mean 4 week
if lag_1week == 1:
    df['lag_1week'] = df['target'].shift(168)

if lag_2week_mean == 1:
    df['lag_2week_mean'] = (
        df['target'].shift(168) + 
        df['target'].shift(2*168)
    ) / 2

if lag_4week_mean == 1:
    df['lag_4week_mean'] = (
        df['target'].shift(168) + 
        df['target'].shift(2*168) +
        df['target'].shift(3*168) +
        df['target'].shift(4*168)
    ) / 4

# Function to add shifted rolling mean features ---
def add_rolling_mean_shifted(df, column, shift_hours, window_hours, name):
    
    df[name] = df[column].shift(shift_hours).rolling(window=window_hours).mean()
    return df

# Add Shifted Rolling Means (all shifted 1 week back)
rolling_configs = [
    (168, 24, 'rolling_1day_shifted'),
    (168, 168, 'rolling_1week_shifted'),
    (168, 336, 'rolling_2week_shifted'),
    (168, 502, 'rolling_3week_shifted'),
    (168, 672, 'rolling_4week_shifted'),
    (168, 1440, 'rolling_2month_shifted'),
    (168, 2160, 'rolling_3month_shifted'),
    (168, 8760, 'rolling_1year_shifted')
]

for shift, window, name in rolling_configs:
    df = add_rolling_mean_shifted(df, column='target', shift_hours=shift, window_hours=window, name=name)

In [ ]:
# features to be scaled
columns_to_scale = ['ghi', 'rain', 'temperature', 'wind_speed_100m','wind_speed_10m','lag_1week', 'lag_2week_mean', 'lag_4week_mean',
                            'rolling_1day_shifted','rolling_1week_shifted', 'rolling_2week_shifted', 'rolling_3week_shifted', 'rolling_4week_shifted',
                              'rolling_2month_shifted', 'rolling_3month_shifted', 'rolling_1year_shifted']

#### Rolling Window and Model

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler

data = df.copy()

test_start_date = pd.Timestamp(test_start_date)
test_size = test_size
n_splits = n_splits

train_data = data.loc[:test_start_date]

results = pd.DataFrame(index=data.loc[test_start_date:].index)
results["target"] = data["target"].reindex(results.index)

# Loop over quantiles
for i, quantile in enumerate(quantiles):
    train_window = train_windows[f"q{i + 1}_train_window"]
    predictions = []
    
    for split in range(n_splits):
        test_start = test_start_date + pd.Timedelta(hours=split * test_size)
        test_end = test_start + pd.Timedelta(hours=test_size - 1)
        
        train_end = test_start - pd.Timedelta(hours=1)
        train_start = max(train_end - pd.Timedelta(hours=train_window), data.index[0])
        
        train_data = data.loc[train_start:train_end]
        test_data = data.loc[test_start:test_end]

        X_train, y_train = train_data.drop(columns=["target"]), train_data["target"]
        X_test, y_test = test_data.drop(columns=["target"]), test_data["target"]

        scaler = StandardScaler()

        X_train[columns_to_scale] = scaler.fit_transform(X_train[columns_to_scale])
        X_test[columns_to_scale] = scaler.transform(X_test[columns_to_scale])

        X_train = pd.DataFrame(X_train, columns=train_data.drop(columns=["target"]).columns, index=train_data.index)
        X_test = pd.DataFrame(X_test, columns=test_data.drop(columns=["target"]).columns, index=test_data.index)

        X_train = sm.add_constant(X_train, has_constant="add")
        y_train = np.asarray(y_train, dtype=float)

        X_test = sm.add_constant(X_test, has_constant="add")

        X_test = X_test[X_train.columns]

        model = sm.QuantReg(y_train, X_train)
        result = model.fit(q=quantile)

        y_pred = result.predict(X_test)
             
        pred_df = pd.DataFrame({
            "index": test_data.index,
            f"q{quantile}": y_pred
        }).set_index("index")
        
        results.loc[pred_df.index, f"q{quantile}"] = pred_df[f"q{quantile}"]



c:\Anaconda\lib\site-packages\statsmodels\regression\quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
c:\Anaconda\lib\site-packages\statsmodels\regression\quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
c:\Anaconda\lib\site-packages\statsmodels\regression\quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
c:\Anaconda\lib\site-packages\statsmodels\regression\quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
c:\Anaconda\lib\site-packages\statsmodels\regression\quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (10

In [ ]:
results.dropna(subset=[col for col in results.columns if col.startswith("q")], inplace=True)

# sort quantile columns if quantile crossing occurs
def fix_quantile_crossing(results):
    
    quantile_columns = [col for col in results.columns if col.startswith('q')]
    
    for idx in results.index:
        sorted_values = sorted(results.loc[idx, quantile_columns].values)
        results.loc[idx, quantile_columns] = sorted_values
    
    return results

In [ ]:
# safe results

folder = "results"
os.makedirs(folder, exist_ok=True)
results.to_csv(f"{folder}/{model_name}.csv", index=True)

#### Evaluation

In [ ]:
# calucate quantile losses of all predictions
quantile_losses = {}

for q in quantiles:
    
    y_pred = results[f'q{q}']
    y_true = results['target']
    
    # pinball loss function multiplied by 2
    quantile_loss = np.where(y_pred > y_true, 
                             2 * (1 - q) * (y_pred - y_true), 
                             2 * q * (y_true - y_pred))
    
    quantile_losses[f'Quantile_{q}'] = quantile_loss.mean()

# losses of all quantile
total_loss_score = sum(quantile_losses.values())

# show results
print("Average loss by quantile:")
for quantile, loss in quantile_losses.items():
    print(f"{quantile}: {loss}")

print(f"\nTotal loss score over all quantiles: {total_loss_score}")

Average loss by quantile:
Quantile_0.025: 0.4084556237556017
Quantile_0.25: 1.7523697636665803
Quantile_0.5: 2.030558016220317
Quantile_0.75: 1.6006949520202676
Quantile_0.975: 0.33473164224456675

Total loss score over all quantiles: 6.126809997907333


In [ ]:
# filter only relevant target horizons
results['hour'] = results.index.hour
results['dayofweek'] = results.index.dayofweek

horizons_dict = {}

# target horizons mapping with dayofweek and hour
target_horizons = [
    {"dayofweek": 4, "hour": 12, "name": "36"},  # Freitag 12:00 Stunde: 36
    {"dayofweek": 4, "hour": 16, "name": "40"},  # Freitag 16:00 Stunde: 40
    {"dayofweek": 4, "hour": 20, "name": "44"},  # Freitag 20:00 Stunde: 44
    {"dayofweek": 5, "hour": 12, "name": "60"},  # Samstag 12:00 Stunde: 60
    {"dayofweek": 5, "hour": 16, "name": "64"},  # Samstag 16:00 Stunde: 64
    {"dayofweek": 5, "hour": 20, "name": "68"},  # Samstag 20:00 Stunde: 68
]

# filter results for target horizons
for horizon in target_horizons:
    horizon_data = results[(results["dayofweek"] == horizon["dayofweek"]) & (results["hour"] == horizon["hour"])]
    horizon_data = horizon_data.drop(columns=["hour", "dayofweek"])

    horizons_dict[horizon["name"]] = horizon_data

In [ ]:
# quantile losses target horizons
def calculate_quantile_losses(horizons_dict, quantiles):
    all_quantile_losses = {}
    
    for key, df in horizons_dict.items():
        quantile_losses = {}
        for q in quantiles:
            y_pred = df[f'q{q}']
            y_true = df['target']
            quantile_loss = np.where(y_pred > y_true, 2 * (1 - q) * (y_pred - y_true), 2 * q * (y_true - y_pred))
            quantile_losses[f'q{q}'] = quantile_loss.mean()
        
        total_loss_score = sum(quantile_losses.values())
        quantile_losses['Total_Loss_Score'] = total_loss_score
        all_quantile_losses[key] = quantile_losses
    
    return all_quantile_losses

quantile_loss_results = calculate_quantile_losses(horizons_dict, quantiles)

horizon_results_df = pd.DataFrame(quantile_loss_results).T
horizon_results_df

,q0.025,q0.25,q0.5,q0.75,q0.975,Total_Loss_Score
36,0.579402,2.192565,2.321447,1.589466,0.247835,6.930715
40,0.508732,1.969286,2.091021,1.567328,0.285276,6.421643
44,0.387507,1.459369,1.609386,1.215420,0.244327,4.916009
60,0.273792,1.395303,1.702532,1.265522,0.237986,4.875135
64,0.355505,1.390563,1.592173,1.269111,0.196045,4.803397
68,0.381594,1.409658,1.630060,1.183042,0.215989,4.820343


#### Final Evaluation Score

In [ ]:
horizon_results_df.sum()

q0.025               2.486532
q0.25                9.816743
q0.5                10.946620
q0.75                8.089889
q0.975               1.427459
Total_Loss_Score    32.767243
dtype: float64